# Project 1 Neuronal networks

## 1.- MNIST and FMINST datasets

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  #To get figures with high quality!

import numpy as np
import torch
from torch import nn
from torch import optim
import matplotlib.pyplot as plt

In [ ]:
!pip install wandb
import wandb # Weight & Biases library to monitor training and compare models
wandb.login() # API key: 569de9861dbe18fe8888f13ca66e39d2b12934ff

To obtain the MNIST dataset:

In [6]:
### Run this cell

from torchvision import datasets, transforms

# Define a transform to normalize the data
transform = transforms.Compose([transforms.ToTensor(),
                              transforms.Normalize((0.5,), (0.5,)),
                              ])

# Download and load the training  data
trainsetMNIST = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
train_MNIST = torch.utils.data.DataLoader(trainsetMNIST, batch_size=64, shuffle=True)

# Download and load the test data
testsetMNIST = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)
test_MNIST = torch.utils.data.DataLoader(testsetMNIST, batch_size=64, shuffle=True)

To obtain the FMINST dataset:

In [5]:

# Download and load the training data
trainsetFMNIST = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
train_FMNIST = torch.utils.data.DataLoader(trainsetFMNIST, batch_size=64, shuffle=True)

# Download and load the test data
testsetFMNIST = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
test_FMNIST = torch.utils.data.DataLoader(testsetFMNIST, batch_size=64, shuffle=True)

100%|██████████| 26421880/26421880 [00:01<00:00, 13299193.76it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 228995.77it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:01<00:00, 4251159.79it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 4859841.77it/s]

Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



In [ ]:
dataiter = iter(train_MNIST)   #To iterate through the dataset

images, labels = next(dataiter)
print(type(images))
print(images.shape)
print(labels.shape)
print(torch.min(images), torch.max(images))

In [ ]:
dataiter = iter(train_FMNIST)   #To iterate through the dataset

images, labels = next(dataiter)
print(type(images))
print(images.shape)
print(labels.shape)
print(torch.min(images), torch.max(images))

We can see that our images are of size 28 x 28 = 784, so the input layer is of size 784. It is important to see that the range of the values are -1,1 so for the output layer we will use the tanh.

## 3 layers at both encoder/decoder:


In [ ]:
class autoencoder3(nn.Module):
    def __init__(self,projected_dimension):
        super().__init__()

        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 -> projected dimension
        self.encoder = nn.Sequential(
            nn.Linear(784, 300), # preguntar al profesor de que tamaño es cada layer
            nn.ReLU(),
            nn.Linear(300, 150),
            nn.ReLU(),
            nn.Linear(150, projected_dimension),
        )
         
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # projected dimension -> 784
        self.decoder = nn.Sequential(
            nn.Linear(projected_dimension, 150),
            nn.ReLU(),
            nn.Linear(150, 300),
            nn.ReLU(),
            nn.Linear(300, 784),
            nn.Tanh()
        )
 
    def forward(self, image):
        encoded = self.encoder(image)
        decoded = self.decoder(encoded)
        return decoded

Definition of parameters of the model and Peak signal-to-noise ratio (PSNR)

In [ ]:
projected_dimensions = [15,30,50,100]
def psnr(img1, img2): #  Peak signal-to-noise ratio (PSNR)
    mse_criterio = nn.MSELoss()
    mse = mse_criterio(img1, img2)
    max_pixel = 1.0  
    psnr = 20 * torch.log10(max_pixel / torch.sqrt(mse))
    return psnr 

Training the model

In [ ]:
total_runs = 4 # To test each of the projected dimensions
for run in range(total_runs):
  model = autoencoder3(projected_dimensions[run]) # To initialize the model with the projected dimension
  optimizer = optim.SGD(model.parameters(), lr = 0.01)
  wandb.init( # Esto es para que se guarde en la plataforma de wandb y nos evitamos tener que hacerlo manualmente
      # Set the project where this run will be logged
      project="project_1",
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"3_layer_with_PSNR{run}",
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.01,
      "architecture": "linear_autoencoder",
      "dataset": "MNIST",
      "epochs": 10,
      })

    # Aqui es donde empieza realmente el entrenamiento
  epochs = 10
  outputs = []
  for epoch in range(epochs):
    for (img,_) in train_MNIST:
      img = img.reshape(-1,784)
      reconstructed = model.forward(img)
      loss = psnr(reconstructed,img)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    print("Epoch: ", epoch + 1, "PSNR: ", loss.item())
    wandb.log({"projected dimension":projected_dimensions[run], "PSNR": psnr})
    outputs.append((projected_dimensions[run],epoch,img,reconstructed))

  wandb.finish()

Visualization of reconstructed images

In [ ]:
for k in range(0,epochs,4):
  plt.figure(figsize=(9,2))
  plt.gray()
  imgs = outputs[k][2].detach().numpy()
  recon = outputs[k][3].detach().numpy()
  for i, item in enumerate(imgs):
    if i >=9: break
    plt.subplot(2,9,i+1)
    item = item.reshape(-1,28,28)

    plt.imshow(item[0])
  for i, item in enumerate(recon):
    if i >=9: break
    plt.subplot(2,9,i+1+9)
    item = item.reshape(-1,28,28)

    plt.imshow(item[0])



# 5 layers at both encoder/decoder:

In [ ]:
class autoencoder5(nn.Module):
    def __init__(self,projected_dimension):
        super().__init__()

        # Building an linear encoder with Linear
        # layer followed by Relu activation function
        # 784 -> projected dimension
        self.encoder = nn.Sequential(
            nn.Linear(784, 550),
            nn.ReLU(),
            nn.Linear(550, 400),
            nn.ReLU(),
            nn.Linear(400, 300),
            nn.ReLU(),
            nn.Linear(300,200),
            nn.ReLU(),
            nn.Linear(200,projected_dimension)
        )
         
        # Building an linear decoder with Linear
        # layer followed by Relu activation function
        # The Sigmoid activation function
        # outputs the value between 0 and 1
        # projected dimension -> 784
        self.decoder = nn.Sequential(
            nn.Linear(projected_dimension, 200),
            nn.ReLU(),
            nn.Linear(200, 300),
            nn.ReLU(),
            nn.Linear(300, 400),
            nn.ReLU(),
            nn.Linear(400,550),
            nn.ReLU(),
            nn.Linear(550,784),
            nn.Tanh()
        )

 
    def forward(self, image):
        encoded = self.encoder(image)
        decoded = self.decoder(encoded)
        return decoded

In [ ]:
total_runs = 4 # To test each of the projected dimensions
for run in range(total_runs):
  model = autoencoder5(projected_dimensions[run]) # To initialize the model with the projected dimension
  optimizer = optim.SGD(model.parameters(), lr = 0.01)
  wandb.init( # Esto es para que se guarde en la plataforma de wandb y nos evitamos tener que hacerlo manualmente
      # Set the project where this run will be logged
      project="project_1", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"5_layer_{run}", 
      # Track hyperparameters and run metadata
      config={
      "learning_rate": 0.01,
      "architecture": "linear_autoencoder",
      "dataset": "MNIST",
      "epochs": 10,
      })

    # Aqui es donde empieza realmente el entrenamiento
  epochs = 10
  outputs = []
  for epoch in range(epochs):
    for (img,_) in train_MNIST:
      img = img.reshape(-1,784) 
      reconstructed = model.forward(img)
      loss = criterion(reconstructed,img)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()

    print("Epoch: ", epoch + 1, "Loss: ", loss.item())
    wandb.log({"projected dimension" :projected_dimensions[run], "loss": loss})
    outputs.append((projected_dimensions[run],epoch,img,reconstructed))

  wandb.finish()

## Regularization:

We will use early stop and drop out. To tune the drop out probability see: https://www.youtube.com/watch?v=9zrmUIlScdY Preguntar en clase si hay q hacer tuning del lr???